In [ ]:
%pylab inline
import numpy as np
from sklearn import preprocessing
from keras.datasets import mnist
from keras.utils import np_utils
np.set_printoptions(suppress=True)
(X_train, y_train), (X_test, y_test) = mnist.load_data()

def preproc(X_train, y_train):
    X = X_train
    X = X.reshape(X.shape[0], X.shape[1] * X.shape[2]).astype("float32")
    #X = X/255
    X = preprocessing.scale(X)
    Y = np_utils.to_categorical(y_train, 10)
    return X, Y

X, Y = preproc(X_train, y_train)
Xt, Yt = preproc(X_test, y_test)

In [ ]:
np.random.seed(1337)
init = 0.01
W1 = np.random.uniform(size=(784, 256),low=-init, high=init).astype(np.float32)
W2 = np.random.uniform(size=(256, 32), low=-init, high=init).astype(np.float32)
W3 = np.random.uniform(size=(32, 256),low=-init, high=init).astype(np.float32)
W4 = np.random.uniform(size=(256, 784), low=-init, high=init).astype(np.float32)

B1 = np.random.uniform(size=(256),low=-init, high=init).astype(np.float32)
B2 = np.random.uniform(size=(32),low=-init, high=init).astype(np.float32)
B3 = np.random.uniform(size=(256),low=-init, high=init).astype(np.float32)
B4 = np.random.uniform(size=(784),low=-init, high=init).astype(np.float32)

In [ ]:
# add biases?
def update(y0):
    global W1, W2, W3, W4
    global B1, B2, B3, B4
    
    lr = 0.001
    def dtanh(x):
        # pass in tanh
        return 1 - (x**2)
    
    # forward pass
    y1 = np.tanh(np.dot(y0, W1) + B1)
    y2 = np.tanh(np.dot(y1, W2) + B2)
    y3 = np.tanh(np.dot(y2, W3) + B3)
    y4 = np.dot(y3, W4) + B4

    # squared error derivative, (target - computed)
    e4 = 2*(y0-y4)
    
    # backward pass, these should be net_i but dtanh is haxx
    e3 = np.dot(e4, W4.T) * dtanh(y3)
    e2 = np.dot(e3, W3.T) * dtanh(y2)
    e1 = np.dot(e2, W2.T) * dtanh(y1)
    #e0 = np.dot(e1, W1.T) * y0
    
    # update the weights
    W4 += np.outer(y3, e4) * lr
    W3 += np.outer(y2, e3) * lr
    W2 += np.outer(y1, e2) * lr
    W1 += np.outer(y0, e1) * lr
    
    # update the biases
    B4 += e4 * lr
    B3 += e3 * lr
    B2 += e2 * lr
    B1 += e1 * lr

    return np.mean((y0-y4)**2)

# run an epoch
err = []
for i in range(X.shape[0]):
    err.append(update(X[i]))
    if np.isnan(err[-1]):
        print "FAILED AT",i, err[-10:]
        break
    if i % 1000 == 999:
        print "%4d: %f" % (i+1, np.mean(err))